# Markov State Models

In [1]:
import torch
import matplotlib.pyplot as plt
from global_utils import CommittorNet, mpath

In [2]:
run_name = "wells_square_0_a0_b3_K2"

In [3]:
net2 = CommittorNet(dim=2,K=2)
net2.load_state_dict(torch.load(mpath(f"{run_name}.pt")))
running_xs_all = torch.load(mpath(f"{run_name}_rxs.pt"))
out_means_k = torch.load(mpath(f"{run_name}_k.pt"))
centers_k = torch.load(mpath(f"{run_name}_ctrs.pt"))